In [1]:
import pandas as pd

In [408]:
import networkx as nx
from IPython.display import IFrame, display
import numpy as np

__TEMPLATE__ = """
<!DOCTYPE html>
<meta charset="utf-8">
<style>
                circle {{
                  fill: #ccc;
                  stroke: #333;
                  stroke-width: 1.5px;
                }}

                .circle.source_node {{
                  fill: yellow;
                }}

                .circle.nice_node {{
                  fill: green;
                }}

                .circle.bad_node {{
                  fill: red;
                }}

                .link {{
                  fill: none;
                  stroke: #666;
                  stroke-opacity: 0.7;
                }}

                #nice_target {{
                  fill: green;
                }}

                .link.nice_target {{
                  stroke: green;
                }}

                #source {{
                  fill: yellow;
                }}

                .link.source {{
                  stroke: yellow;
                }}

                #bad_target {{
                  fill: red;
                }}

                .link.bad_target {{
                  stroke: red;
                }}
text {{
  font: 10px sans-serif;
  pointer-events: none;
  text-shadow: 0 1px 0 #fff, 1px 0 0 #fff, 0 -1px 0 #fff, -1px 0 0 #fff;
}}

</style>
<body>
<script src="//d3js.org/d3.v4.min.js"></script>
<script>

// http://blog.thomsonreuters.com/index.php/mobile-patent-suits-graphic-of-the-day/
var links = {links};
var node_params = {node_params};

var nodes = {nodes};

var width = {width},
    height = {height};

var svg = d3.select("body").append("svg")
    .attr("width", width)
    .attr("height", height);

// Per-type markers, as they don't inherit styles.
svg.append("defs").selectAll("marker")
    .data(["source", "nice_target", "bad_target", "suit", "end"])
  .enter().append("marker")
    .attr("id", function(d) {{ return d; }})
    .attr("viewBox", "0 -5 10 10")
    .attr("refX", 15)
    .attr("refY", -1.5)
    .attr("markerWidth", 10)
    .attr("markerHeight", 10)
    .attr("markerUnits", "userSpaceOnUse")
    .attr("orient", "auto")
  .append("path")
    .attr("d", "M0,-5L10,0L0,5");

var path = svg.append("g").selectAll("path")
    .data(links)
  .enter().append("path")
    .attr("class", function(d) {{ return "link " + d.type; }})
    .attr("stroke-width", function(d) {{ return (d.weight * 10); }})
    .attr("marker-end", function(d) {{ return "url(#" + d.type + ")"; }})
    .attr("id", function(d,i) {{ return "link_"+i; }})
    .attr("d", linkArc)
    ;

var edgetext = svg.append("g").selectAll("text")
    .data(links)
   .enter().append("text")
   .append("textPath")
    .attr("xlink:href",function(d,i){{return "#link_"+i;}})
    .style("text-anchor","middle")
    .text(function(d){{ return Math.round(d.weight * 100) + "%"; }})
    .attr("startOffset", "50%")
    ;

function dragstarted(d) {{
  d3.select(this).raise().classed("active", true);
}}

function dragged(d) {{
  d3.select(this).attr("cx", d.x = d3.event.x).attr("cy", d.y = d3.event.y);
}}

function dragended(d) {{
  d3.select(this).classed("active", false);
  path = path.attr("d", linkArc);
  text = text
        .attr('x', function(d) {{ return d.x; }})
        .attr('y', function(d) {{ return d.y; }})
        ;
}};

var circle = svg.append("g").selectAll("circle")
    .data(nodes)
  .enter().append("circle")
    .attr("class", function(d) {{ return "circle " + d.type; }})
    .attr("r", 6)
    .attr('cx', function(d) {{ return d.x; }})
    .attr('cy', function(d) {{ return d.y; }})
    .call(d3.drag()
        .on("start", dragstarted)
        .on("drag", dragged)
        .on("end", dragended));

var text = svg.append("g").selectAll("text")
    .data(nodes)
  .enter().append("text")
    .attr('x', function(d) {{ return d.x; }})
    .attr('y', function(d) {{ return d.y; }})
    .text(function(d) {{ return d.name; }});

function linkArc(d) {{
  var dx = nodes[d.target.index].x - nodes[d.source.index].x,
      dy = nodes[d.target.index].y - nodes[d.source.index].y,
      dr = Math.sqrt(dx * dx + dy * dy);
      if (dr > 0) {{return "M" + nodes[d.source.index].x + "," + nodes[d.source.index].y + "A" + dr + "," + dr + " 0 0,1 " + nodes[d.target.index].x + "," + nodes[d.target.index].y;}}
      else {{return "M" + nodes[d.source.index].x + "," + nodes[d.source.index].y + "A" + 20 + "," + 20 + " 0 1,0 " + (nodes[d.target.index].x + 0.1) + "," + (nodes[d.target.index].y + 0.1);}}
}}
</script>
"""


# def calc_layout(data):
#     G = nx.DiGraph()
#     G.add_weighted_edges_from(data.loc[:, ['source', 'target', 'weight']].values)
#     pos = sugiyama_layout(G)
#     return pos


def calc_layout(data, node_params, width=500, height=500):
    inverse_node_params = {j: i for i, j in node_params.items()}
    pos = {
        inverse_node_params.get('source'): np.array([50 / width, 1 / 2]),
        inverse_node_params.get('nice_target'): np.array([1 - 50 / width, 1 / 3]),
        inverse_node_params.get('bad_target'): np.array([1 - 50 / width, 2 * 1 / 3]),
    }
    no_poi = data[~(data.source.isin(node_params) | data.target.isin(node_params))].copy()
    G = nx.DiGraph()
    G.add_weighted_edges_from(data.loc[:, ['source', 'target', 'weight']].values)
    pos_new = nx.layout.fruchterman_reingold_layout(G, k=1, pos=pos, fixed=list(node_params.keys() & G.nodes))
    min_x = min([j[0] for i, j in pos_new.items()])
    min_y = min([j[1] for i, j in pos_new.items()])
    max_x = max([j[0] for i, j in pos_new.items()])
    max_y = max([j[1] for i, j in pos_new.items()])
    pos_new = {i: [(j[0] - min_x) / (max_x - min_x) * (width - 150) + 75, (j[1] - min_y) / (max_y - min_y) * (height - 100) + 50] for i, j in pos_new.items()}
    pos_new.update({i: [j[0] * width, j[1] * height] for i, j in pos.items()})
    return pos_new


def prepare_nodes(data, pos, node_params):
    node_set = set(data['source']) | set(data['target'])
    nodes = {}
    for idx, node in enumerate(node_set):
        node_pos = pos.get(node)
        nodes.update({node: {
            "index": idx,
            "name": node,
            "x": node_pos[0],
            "y": node_pos[1],
            "type": (node_params.get(node) or "suit").split('_')[0] + '_node'
        }})
    return nodes


def prepare_edges(data, nodes):
    edges = []
    for idx, row in data.iterrows():
        if nodes.get(row.source).get('type') == 'source':
            edge_type = 'source'
        else:
            edge_type = nodes.get(row.target).get('type')
        edges.append({
            "source": nodes.get(row.source),
            "target": nodes.get(row.target),
            "weight": np.log1p(row.weight),
            "type": row['type']
        })
    return edges, list(nodes.values())


def make_json_data(data, node_params, thresh=.05, width=500, height=500):
    res = {}
    data.columns = ['source', 'target', 'weight']
    data = data[data.weight >= thresh].copy()
    data["type"] = data.apply(
        lambda x: node_params.get(x.source) if node_params.get(x.source) == 'source' else node_params.get(x.target) or 'suit',
        1
    )
    pos = calc_layout(data, node_params, width=width, height=height)
    nodes = prepare_nodes(data, pos, node_params)
    res['links'], res['nodes'] = prepare_edges(data, nodes)
    return res


def plot_graph(data, node_params, thresh=.05, width=500, height=500):
    res = make_json_data(data, node_params, thresh=thresh, width=width - 100, height=height - 100)
    x = __TEMPLATE__.format(
        width=width,
        height=height,
        links=res.get('links'),
        node_params=node_params,
        nodes=res.get('nodes')
    )
    with open('./index.html', 'w') as f:
        f.write(x)
    display(IFrame('./index.html', width=width + 100, height=height + 100))


In [409]:
data = pd.read_csv('/mnt/c/Users/mi/Downloads/Telegram Desktop/datavis/case_14_many.csv')
node_params = {
      "app_remove": "bad_target",
      "chout_link": "nice_target",
      "session_start": "source",
    }
plot_graph(data, node_params, thresh=.05, width=700, height=700)

In [406]:
def calc_layout(data, node_params, width=500, height=500):
    inverse_node_params = {j: i for i, j in node_params.items()}
    inverse_node_params = {j: i for i, j in node_params.items()}
    pos = {
        inverse_node_params.get('source'): np.array([.1, .5]),
        inverse_node_params.get('nice_target'): np.array([.9, .33]),
        inverse_node_params.get('bad_target'): np.array([.9, .67]),
    }
    
    second_layer = data[data.source == inverse_node_params.get('source')].target.values
    prelast_layer = data[data.target.isin([inverse_node_params.get('nice_target'),
                                           inverse_node_params.get('bad_target')])].source.values
    
    for idx, node in enumerate(prelast_layer):
        pos.update({
            node: [0.75, (idx + 1) / (len(prelast_layer) + 1)]
        })

    for idx, node in enumerate(second_layer):
        pos.update({
            node: [0.25, (idx + 1) / (len(second_layer) + 1)]
        })
    
    G = nx.DiGraph()
    G.add_weighted_edges_from(data.loc[:, ['source', 'target', 'weight']].values)
    
    pos_new = nx.layout.spring_layout(G, pos=pos, weight='weight', fixed=list(node_params.keys() & G.nodes), seed=0)
    pos_new = __POS__
    min_x = min([j[0] for i, j in pos_new.items()])
    min_y = min([j[1] for i, j in pos_new.items()])
    max_x = max([j[0] for i, j in pos_new.items()])
    max_y = max([j[1] for i, j in pos_new.items()])
    pos_new = {i: [((j[0] - min_x) / (max_x - min_x) / 2.5 + .3) * width, (j[1] - min_y) / (max_y - min_y) * (height - 100) + 50] for i, j in pos_new.items()}
    pos_new.update({i: [j[0] * width, j[1] * height] for i, j in pos.items()})
    pos_new = {i: [j[0] if not np.isnan(j[0]) else .5 * width, j[1] if not np.isnan(j[1]) else .5 * height] for i, j in pos_new.items()}
    print(pos_new)
    return pos_new